## Import stuff

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
import math
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())



2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14468512894105293593
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


In [12]:
# df = pd.read_csv("train_features.csv")
df = pd.read_csv("test_features.csv")
ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1)
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
ratios = ratios.drop(["Time", "Age"], axis=1)
ratios.columns = [x+"_ratio" for x in ratios.columns]
ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)

#LAST HOUR

# # go through 12 rows at a time
for i in range(0, len(df), 12):
    # get the 12 rows
    patient = df.iloc[i:i+12]
    #fill each column with the last value
    for col in patient.columns:
        mean = patient[col].mean()
        if not math.isnan(mean):
            patient[col] = patient[col].dropna().iloc[-1]

    # print(patient)
    # break

last_hour = pd.DataFrame(np.array(df.iloc[:, 1:].values,  dtype=float)[11::12], columns=df.columns[1:])

#normalize each column except the first
last_hour2 = last_hour.iloc[:,1:]
last_hour2 = last_hour2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
last_hour.iloc[:,1:] = last_hour2

# merge the two dataframes
df = pd.concat([last_hour, ratios], axis=1)

#make "pid" the first column
first_column = df.pop('pid')
df.insert(0, 'pid', first_column)
df.replace([np.inf, -np.inf], 0, inplace=True)
# df.to_csv('train_features_ratio.csv', index=False, float_format='%.3f',na_rep='nan')
df.to_csv('test_features_ratio.csv', index=False, float_format='%.3f',na_rep='nan')

<ipython-input-12-8373bee243de>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].dropna().iloc[-1]


In [13]:
# df = pd.read_csv("train_features_ratio.csv")
df = pd.read_csv("test_features_ratio.csv")
X = np.array(df.iloc[:, 1:].values,  dtype=float)

imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(X)
df_imputed = pd.DataFrame(df_imputed)
df_imputed.insert(0, 'pid', first_column)
df_imputed.columns = df.columns
# df_imputed
# df_imputed.to_csv('train_features_imputed.csv', index=False, float_format='%.3f',na_rep='nan')
df_imputed.to_csv('test_features_imputed.csv', index=False, float_format='%.3f',na_rep='nan')

## the direction in which the values are going could be used as a feature

In [64]:
df = pd.read_csv("train_features.csv")
ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1)
orig_ordering = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
ratios = pd.merge(orig_ordering["pid"], ratios, on="pid")
ratios = ratios.drop(["Time", "Age"], axis=1)
ratios.columns = [x+"_ratio" for x in ratios.columns]
ratios.rename(columns={'pid_ratio': 'pid'}, inplace=True)
ratios

,pid,EtCO2_ratio,PTT_ratio,BUN_ratio,Lactate_ratio,Temp_ratio,Hgb_ratio,HCO3_ratio,BaseExcess_ratio,RRate_ratio,...,Alkalinephos_ratio,SpO2_ratio,Bilirubin_direct_ratio,Chloride_ratio,Hct_ratio,Heartrate_ratio,Bilirubin_total_ratio,TroponinI_ratio,ABPs_ratio,pH_ratio
0,1.0,NaN,NaN,0.000000,NaN,0.055556,-0.022989,0.083333,-1.0,0.125000,...,NaN,0.000000,NaN,-0.026316,-0.097561,-0.265957,NaN,NaN,-0.281690,0.008186
1,10.0,NaN,0.00000,0.000000,NaN,0.000000,0.000000,NaN,NaN,-0.304348,...,0.000000,-0.010101,NaN,NaN,0.000000,-0.085366,0.000000,0.000000,-0.062069,NaN
2,100.0,NaN,0.00000,0.000000,NaN,0.085714,0.000000,0.000000,NaN,-0.071429,...,NaN,-0.030000,NaN,0.000000,0.000000,0.202128,NaN,NaN,-0.265823,NaN
3,1000.0,0.115385,NaN,0.000000,-0.096296,0.055556,0.000000,NaN,NaN,0.000000,...,NaN,-0.050000,NaN,NaN,0.000000,-0.281818,NaN,NaN,0.008065,0.000000
4,10000.0,NaN,0.22179,0.000000,NaN,0.027778,-0.054545,0.040000,inf,-0.272727,...,NaN,-0.010000,NaN,-0.009615,-0.063898,0.406250,NaN,NaN,0.200000,0.006803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,NaN,NaN,0.076923,NaN,-0.027778,0.256757,NaN,NaN,-0.280000,...,-0.030303,0.052632,NaN,NaN,0.179167,-0.164179,-0.166667,0.253165,-0.151261,NaN
18991,9995.0,NaN,0.00000,0.000000,NaN,0.000000,0.000000,0.000000,NaN,0.235294,...,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,-0.005714,NaN
18992,9996.0,NaN,NaN,NaN,NaN,0.027027,-0.018349,NaN,NaN,0.800000,...,NaN,0.000000,NaN,NaN,-0.239394,0.104651,NaN,NaN,-0.159292,0.002699
18993,9998.0,NaN,0.00000,0.000000,NaN,0.027778,0.000000,NaN,NaN,0.052632,...,NaN,0.000000,NaN,NaN,0.000000,-0.056604,NaN,-0.606481,0.431193,NaN


In [103]:
df = pd.read_csv("train_features.csv")
last_hour = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])
# last_hour.sort_values(by=['pid'], inplace=True)
# last_hour = last_hour.set_index("pid")

#normalize each column except the first 2
last_hour2 = last_hour.iloc[:,2:]
last_hour2 = last_hour2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
last_hour.iloc[:,2:] = last_hour2
last_hour

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1.0,14.0,0.223529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,0.153191,0.262821,NaN,NaN,0.331658,NaN
1,10.0,12.0,0.658824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.959184,NaN,NaN,NaN,0.301282,NaN,NaN,0.502513,NaN
2,100.0,13.0,0.623529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.938776,NaN,NaN,NaN,0.544872,NaN,NaN,0.402010,NaN
3,1000.0,12.0,0.752941,0.275362,NaN,NaN,NaN,0.666667,NaN,NaN,...,NaN,0.897959,NaN,NaN,NaN,0.326923,NaN,NaN,0.447236,NaN
4,10000.0,12.0,0.717647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.979592,NaN,NaN,NaN,0.397436,NaN,NaN,0.482412,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,12.0,0.764706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.538462,NaN,NaN,0.326633,NaN
18991,9995.0,12.0,0.682353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.918367,NaN,NaN,NaN,0.230769,NaN,NaN,0.693467,NaN
18992,9996.0,18.0,0.447059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.429487,NaN,NaN,0.296482,NaN
18993,9998.0,12.0,0.870588,NaN,NaN,NaN,NaN,0.555556,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,0.461538,NaN,NaN,0.603015,NaN


In [23]:
df = pd.read_csv("train_features.csv")
# patients = pd.read_csv("test_features.csv")

# go through 12 rows at a time
for i in range(0, len(df), 12):
    # get the 12 rows
    patient = df.iloc[i:i+12]
    #fill each column with the last value
    for col in patient.columns:
        mean = patient[col].mean()
        if not math.isnan(mean):
            # print(col)
            # print(patient[col].dropna().iloc[-1])
            patient[col] = patient[col].dropna().iloc[-1]

    print(patient)
    break

last_hour = pd.DataFrame(np.array(df.iloc[:, :].values,  dtype=float)[11::12], columns=df.columns[:])

#normalize each column except the first 2
last_hour2 = last_hour.iloc[:,2:]
last_hour2 = last_hour2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
last_hour.iloc[:,2:] = last_hour2
last_hour


    pid  Time   Age  EtCO2  PTT   BUN  Lactate  Temp  Hgb  HCO3  ...  \
0     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
1     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
2     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
3     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
4     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
5     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
6     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
7     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
8     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
9     1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
10    1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   
11    1    14  34.0    NaN  NaN  12.0      NaN  38.0  8.5  26.0  ...   

    Alkalinephos   SpO2  Bilirubin_direct  Chloride   Hct  Hear

<ipython-input-23-a738310f5653>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].dropna().iloc[-1]


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1.0,14.0,34.0,NaN,NaN,12.0,NaN,38.0,8.5,26.0,...,NaN,100.0,NaN,111.0,22.2,69.0,NaN,NaN,102.0,7.39
1,10.0,12.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,98.0,NaN,NaN,NaN,75.0,NaN,NaN,136.0,NaN
2,100.0,13.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.0,NaN,NaN,NaN,113.0,NaN,NaN,116.0,NaN
3,1000.0,12.0,79.0,29.0,NaN,NaN,NaN,38.0,NaN,NaN,...,NaN,95.0,NaN,NaN,NaN,79.0,NaN,NaN,125.0,NaN
4,10000.0,12.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,99.0,NaN,NaN,NaN,90.0,NaN,NaN,132.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993.0,12.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,112.0,NaN,NaN,101.0,NaN
18991,9995.0,12.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.0,NaN,NaN,NaN,64.0,NaN,NaN,174.0,NaN
18992,9996.0,18.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,95.0,NaN,NaN,95.0,NaN
18993,9998.0,12.0,89.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,100.0,NaN,NaN,156.0,NaN


## code to remove nan values (obsolete)

In [6]:
patients = pd.read_csv("train_features.csv")
# patients = pd.read_csv("test_features.csv")


#get the mean for each column
mean_values = patients.mean()


# go through 12 rows at a time
for i in range(0, len(patients), 12):
    # get the 12 rows
    patient = patients.iloc[i:i+12]
    #for each column, use linear interpolation to replace the nan values
    for col in patient.columns:
        patient[col] = patient[col].interpolate(method='linear')
        # mean = patient[col].mean()
        # if math.isnan(mean):
        #     patient[col] = mean_values[col]
        mean = patient[col].mean()
        patient[col] = patient[col].fillna(mean)



    if i%1000 == 0:
        print(i)
        # break
    # break



#save the dataframe
# patients.to_csv("train_features_fixed.csv", index=False, float_format='%.3f',na_rep='nan')
patients.to_csv("train_features_fixed_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')
# patients.to_csv("test_features_fixed_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')




<ipython-input-6-8cddb51b224d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].interpolate(method='linear')
<ipython-input-6-8cddb51b224d>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].fillna(mean)


0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
132000
135000
138000
141000
144000
147000
150000
153000
156000
159000
162000
165000
168000
171000
174000
177000
180000
183000
186000
189000
192000
195000
198000
201000
204000
207000
210000
213000
216000
219000
222000
225000


# (obsolete)

In [7]:
# patients = pd.read_csv("train_features_fixed.csv")
# patients = pd.read_csv("test_features_fixed.csv")
patients = pd.read_csv("train_features_fixed_withnancols.csv")

#normalize each column except the first 2
patients2 = patients.iloc[:,2:]
patients2 = patients2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
patients.iloc[:,2:] = patients2


#save the dataframe
patients.to_csv("train_features_normalized_withnancols.csv", index=False, float_format='%.3f',na_rep='nan')